# Let's build a cross encoder - Step by step guide

We'll use it for the STS task. We'll use the pretrained BERT model for transfer learning on this new semantic sim task.

In [ ]:
!pip install -q transformers pytorch_lightning
!wget -q https://raw.githubusercontent.com/dumitrescustefan/RO-STS/master/dataset/text-similarity/RO-STS.train.tsv
!wget -q https://raw.githubusercontent.com/dumitrescustefan/RO-STS/master/dataset/text-similarity/RO-STS.dev.tsv
!wget -q https://raw.githubusercontent.com/dumitrescustefan/RO-STS/master/dataset/text-similarity/RO-STS.test.tsv

import logging, os, sys, json, torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
import pytorch_lightning as pl
from transformers import AutoTokenizer, AutoModel, AutoConfig, Trainer, TrainingArguments
from pytorch_lightning.callbacks import EarlyStopping
import numpy as np

# we'll define or model name here
transformer_model_name = "dumitrescustefan/bert-base-romanian-cased-v1"

## Data loading

In [ ]:
# before writting any code we're going to need our tokenizer:
tokenizer = AutoTokenizer.from_pretrained(transformer_model_name, strip_accents=False)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, tokenizer, file):
        self.tokenizer = tokenizer  # we'll need this in the __getitem__ function
        self.instances = []
        with open(file, "r", encoding="utf8") as f:
            lines = f.readlines()
            for line in lines:
              parts = line.split("\t")
              self.instances.append({
                  "sim": float(parts[0])/5., 
                  "sent": f"[CLS]{parts[1].strip()}[SEP]{parts[2].strip()}[SEP]"
                  })

    def __len__(self):
        return len(self.instances)  # return how many instances we have. It's a list after all

    def __getitem__(self, index):
        return self.instances[index]

Let's test it's working. Load a dataset and print the first example.

In [ ]:
# create the MyDataset object with the test_data
test_dataset = MyDataset(tokenizer, "RO-STS.test.tsv")
instance = test_dataset[0]  # this calls our __getitem__(0) method

# now let's print what it contains:
for key in instance:
  print(f"{key}: {instance[key]}")

Now, we need to collate the instances in a batch.

In [ ]:
class MyCollator(object):
    def __init__(self, tokenizer, max_seq_len):
        self.max_seq_len = max_seq_len  # this will be our model's maximum sequence length
        self.tokenizer = tokenizer   # we still need our tokenizer to know that the pad token's id is
             

    def __call__(self, input_batch):
        sims = []
        sents = []

        for instance in input_batch:
          sims.append(instance['sim'])
          sents.append(instance['sent'])

        tokenized_batch = self.tokenizer(sents, padding=True, max_length = self.max_seq_len, truncation=True, return_tensors="pt")
        sims = torch.tensor(sims, dtype=torch.float)

        return {
            "tokenized_batch": tokenized_batch,
            "sim": sims
        }

In [ ]:
# let's test our collator
test_dataset = MyDataset(tokenizer, "RO-STS.train.tsv")
my_collator = MyCollator(tokenizer=tokenizer, max_seq_len=512)

# crete a dataloader and get first batch of 3
test_dataloader = DataLoader(test_dataset, batch_size=3, collate_fn=my_collator)

iterable_data = iter(test_dataloader)
first_batch = next(iterable_data) # this is the output_batch from above
for key in first_batch:
  print(f"{key} is a {first_batch[key]}")

## Model preparation

We're finally here :)

As we're using Pytorch Lightning to do the behind-the-scenes training, we do need to define a few functions: 

* ``__init__``, ``forward``
* ``training_step``
* ``validation_step``
* ``configure_optimizers``

As this is a single block of code, comments will be inline:


In [ ]:
class TransformerModel(pl.LightningModule):
    def __init__(self, model_name, lr=2e-05, model_max_length=512):
        super().__init__()

        print("Loading AutoModel [{}] ...".format(model_name))

        self.tokenizer = AutoTokenizer.from_pretrained(model_name, strip_accents=False)
        self.model = AutoModel.from_pretrained(model_name)
        self.output_layer = torch.nn.Linear(768,1)
        
        self.loss_fct = torch.nn.MSELoss()   
        
        self.lr = lr
      
    def forward(self, tokenized_batch):
        # we're just wrapping the code on the AutoModelForTokenClassification
        # it needs the input_ids, attention_mask and labels

        output = self.model(
            input_ids=tokenized_batch['input_ids'],
            attention_mask=tokenized_batch['attention_mask'],
            return_dict=True
        )
        pooler_output = output['pooler_output']  # [batch_size, 768]
        prediction = self.output_layer(pooler_output)  # [batch_size, 1]

        return prediction.flatten()
        

    def training_step(self, batch, batch_idx):
        tokenized_batch = batch['tokenized_batch']
        sims = batch['sim']

        prediction = self.forward(tokenized_batch)  # [batch_size, 1]
        
        loss = self.loss_fct(prediction, sims)

        self.log("train_loss", loss.detach().cpu().item(), on_step=True, on_epoch=True, prog_bar=True,)
        return {"loss": loss}


    def validation_step(self, batch, batch_idx):
        tokenized_batch = batch['tokenized_batch']
        sims = batch['sim']

        prediction = self.forward(tokenized_batch)  # [batch_size, seq_len, 768]
       
        loss = self.loss_fct(prediction, sims)

        self.log("val_loss", loss.detach().cpu().item(), on_step=True, on_epoch=True, prog_bar=True,)
        return {"loss": loss}

    def configure_optimizers(self):
        return torch.optim.AdamW([p for p in self.parameters() if p.requires_grad], lr=self.lr, eps=1e-08)

## Training phase

At this point we're ready to start training. When the code is ready, switch your colab to GPU, and run every cell up to this point, to have the training run on the GPU. Notice that Pytorch Lightning abstracts all the hassle of training on different devices. 

So, what do we need?

We need the model itself (the ``TransformerModel`` object), and the trainer object which receives a few parameters detailed below. The trainer will move the data on GPU automatically, call ``train_step`` and ``train_epoch_end``, then do the same for validation, and then do backprop (internally calls Pytorch's ``.backward()``, ``optimizer_step`` and ``zero_grad`` to update the model weights. It also handles all the gritty stuff like early stopping, logging, model saving, distributed training (if you have more than 1 GPU), etc.


In [ ]:
model = TransformerModel(
    model_name=transformer_model_name,
    lr=2e-5,
    model_max_length=512
)

trainer = pl.Trainer(
    devices=-1,  # uncomment this when training on gpus
    accelerator="gpu",  # uncomment this when training on gpus
    max_epochs=2,  # set this to -1 when training fully 
    #limit_train_batches=10,  # comment this out when training fully
    #limit_val_batches=5,  # comment this out when training fully
    gradient_clip_val=1.0,
    enable_checkpointing=False  # this disables saving the model each epoch
)

# instantiate dataloaders
# a batch_size of 8 should work fine on 16GB GPUs
train_dataloader = DataLoader(MyDataset(tokenizer, "RO-STS.train.tsv"), batch_size=8, collate_fn=my_collator, shuffle=True, pin_memory=True)
validation_dataloader = DataLoader(MyDataset(tokenizer, "RO-STS.dev.tsv"), batch_size=8, collate_fn=my_collator, shuffle=False, pin_memory=True)

# call this to start training
trainer.fit(model, train_dataloader, validation_dataloader)

## Let's use our model

In [ ]:
def predict (model, sent1, sent2):
    # prepare a string with the concatenated sentences

    # tokenize the sentence, don't forget to make it an 1-element list 
    
    # run through the model, it returns a [batch_size] prediction
    
    return # return the value that you multiply by 5

### Solution (hidden)

In [ ]:
def predict (model, sent1, sent2):
    concatenated_sentences = f"[CLS]{sent1.strip()}[SEP]{sent2.strip()}[SEP]"

    tokenized_batch = model.tokenizer([concatenated_sentences], padding=True, max_length = 512, truncation=True, return_tensors="pt")
    
    predictions = model.forward(tokenized_batch)  # returns a [batch_size, ]
    
    return predictions[0].item()*5.  # select the first item and multiply by 5

## Evaluation

In [ ]:
# let's test our code
model.eval()

tests = {
    ("Ana are mere.", "Andrei are pere."),
    ("Ana are mere.", "Merg la munte pe un drum."),
    ("Filmul este foarte bun", "Filmul este extrem de slab.")
}

for (s1, s2) in tests:
  print(f"{s1} || {s2} \t SIM = {predict(model, s1, s2)}")